# Implementing models with Keras

## AlexNet

In [29]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np

np.random.seed(1000)

AlexNet = Sequential()

# 1 CONV LAYER
AlexNet.add(Conv2D(filters=96, input_shape=(32, 32, 3), kernel_size=(11, 11), strides=(4, 4), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

# 2 CONV LAYER
AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

# 3 CONV LAYER
AlexNet.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

# 4 CONV LAYER
AlexNet.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

# 5 CONV LAYER
AlexNet.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

# FC LAYERS
AlexNet.add(Flatten())
# 1st FC LAYER
AlexNet.add(Dense(4096,input_shape=(32, 32, 3, )))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
# DROP OUT
AlexNet.add(Dropout(0.4))

# 2nd FC LAYER
AlexNet.add(Dense(4096))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

# ADD DROPOUT
AlexNet.add(Dropout(0.4))

# 3rd FC LAYER
AlexNet.add(Dense(1000))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
# ADD DROPOUT
AlexNet.add(Dropout(0.4))

# OUTPUT LAYER
AlexNet.add(Dense(7))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('softmax'))

# Model Summary
AlexNet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 8, 8, 96)          34944     
_________________________________________________________________
batch_normalization_10 (Batc (None, 8, 8, 96)          384       
_________________________________________________________________
activation_10 (Activation)   (None, 8, 8, 96)          0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 4, 4, 96)          0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 4, 4, 256)         614656    
_________________________________________________________________
batch_normalization_11 (Batc (None, 4, 4, 256)         1024      
_________________________________________________________________
activation_11 (Activation)   (None, 4, 4, 256)         0         
__________

In [31]:
AlexNet.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])

## Loading data

In [22]:
# lets count data
import os
from tensorflow import keras

train_data = 'Datasets/Propio/primer_grupo/train_data'
test_data = 'Datasets/Propio/primer_grupo/test_data'
validation_data = 'Datasets/Propio/primer_grupo/validation_data'

train_ds = keras.preprocessing.image


In [26]:
from keras.preprocessing.image import ImageDataGenerator
im_width, im_height = 32, 32
nb_train_samples = 8144
nb_validation_samples = 8041
epochs = 10
batch_size = 32
n_classes = 7

datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    rotation_range=5,
    horizontal_flip=True
)
train_gen = datagen.flow_from_directory(train_data, target_size=(im_width, im_height), class_mode='categorical', batch_size=batch_size)
validation_gen = datagen.flow_from_directory(validation_data, target_size=(im_width, im_height), class_mode='categorical', batch_size=batch_size)


Found 489 images belonging to 7 classes.
Found 105 images belonging to 7 classes.


In [27]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
early_stop = EarlyStopping(monitor='val_loss', patience=8, verbose=1, min_delta=1e-4)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, min_delta=1e-4)
callbacks_list = [early_stop, reduce_lr]

In [32]:
model_history = AlexNet.fit_generator(
    train_gen,
    epochs=epochs,
    validation_data=validation_gen,
    validation_steps=nb_validation_samples//batch_size,
    callbacks=callbacks_list
)

Epoch 1/10
16/16 [==============================] - 45s 3s/step - loss: 1.5242 - acc: 0.4442 - val_loss: 13.8148 - val_acc: 0.1429
Epoch 2/10
16/16 [==============================] - 35s 2s/step - loss: 0.9700 - acc: 0.7105 - val_loss: 13.2471 - val_acc: 0.1429
Epoch 3/10
16/16 [==============================] - 34s 2s/step - loss: 0.8053 - acc: 0.8073 - val_loss: 7.7570 - val_acc: 0.2567
Epoch 4/10
16/16 [==============================] - 34s 2s/step - loss: 0.6936 - acc: 0.8261 - val_loss: 2.9694 - val_acc: 0.2678
Epoch 5/10
16/16 [==============================] - 35s 2s/step - loss: 0.6266 - acc: 0.8574 - val_loss: 2.5448 - val_acc: 0.2834
Epoch 6/10
16/16 [==============================] - 37s 2s/step - loss: 0.5727 - acc: 0.8997 - val_loss: 2.9139 - val_acc: 0.2870
Epoch 7/10
16/16 [==============================] - 38s 2s/step - loss: 0.5250 - acc: 0.9150 - val_loss: 5.9642 - val_acc: 0.1426
Epoch 8/10
16/16 [==============================] - 37s 2s/step - loss: 0.5242 - acc: 0.

In [ ]:
from matplotlib import pyplot as plt
plt.figure(0)
plt.plot(model_history.history['acc'], 'r')
plt.plot(model_history.history['val_acc'], 'g')
plt.xticks(np.arange(0, 20, 1.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of epochs")
plt.ylabel("Accuracy")
plt.title("Training Accuracy vs Validation Accuracy")
plt.legend(['train', 'validation'])

plt.figure(1)
plt.plot(model_history.history['loss'], 'r')
plt.plot(model_history.history['val_loss'], 'g')
plt.xticks(np.arange(0, 20, 1.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of epochs")
plt.ylabel("Loss")
plt.title("Training Loss vs Validation Loss")
plt.legend(['train', 'validation'])

In [ ]:
AlexNet.evaluate_generator(validation_gen, steps=None, max_queue_size=10, workers=1, use_multiprocessing=True)

In [ ]:
pred = AlexNet.predict_generator(validation_gen, steps=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=1)
predicted = np.argmax(pred, axis=1)

In [1]:
import os
from utils.ReID import extract_masks, apply_mask, dpm, lbp

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


In [1]:
path = 'Datasets/Propio/1/cuarto'
for image_path in os.listdir(path):
    root = os.path.join(path, image_path)
    frame = cv2.imread(root)
    scale = 10
    width = int(frame.shape[1] * scale / 100)
    height = int(frame.shape[0] * scale / 100)
    frame = cv2.resize(frame, (width, height))
    frame_cp = frame.copy()
    r, output = extract_masks(frame)
    if len(r["rois"]) != 0 and len(r["masks"]) != 0:
        for i in range(len(r["rois"])):
            mask = r["masks"][:, :, i].astype(int)
            temp_frame = apply_mask(frame_cp, mask)
            h_crop, t_crop, l_crop = dpm(r["rois"][i], temp_frame)
            head_hg = lbp(h_crop)
            torso_hg = lbp(t_crop)
            legs_hg = lbp(l_crop)


001__8 01.jpg
001__8 02.jpg
001__8 03.jpg
001__8 04.jpg
001__8 05.jpg
001__8 06.jpg
001__8 07.jpg
001__8 08.jpg
001__8 09.jpg
001__8 10.jpg
001__8 11.jpg
001__8 12.jpg
001__8 13.jpg
001__8 14.jpg
001__8 15.jpg
001__8 16.jpg
001__8 17.jpg
001__8 18.jpg
001__8 19.jpg
001__8 20.jpg
001__8 21.jpg
001__8 22.jpg
001__8 23.jpg
001__8 24.jpg
001__8 25.jpg
001__8 26.jpg
001__8 27.jpg
001__8 28.jpg
001__8 29.jpg
001__8 30.jpg
001__8 31.jpg
001__8 32.jpg
001__8 33.jpg
001__8 34.jpg
001__8 35.jpg
001__8 36.jpg
001__8 37.jpg
001__8 38.jpg
001__8 39.jpg
001__8 40.jpg
001__8 41.jpg
001__8 42.jpg
001__8 43.jpg
001__8 44.jpg
001__8 45.jpg
001__8 46.jpg
001__8 47.jpg
001__8 48.jpg
001__8 49.jpg
001__8 50.jpg
001__8 51.jpg


In [11]:
from utils.ReID import extract_masks, calc_lbph, dpm, apply_mask, crop_frame, lbp

In [14]:
import os
import cv2

parent_root = 'Datasets/Propio/primer_grupo/test_data'

for person in os.listdir(parent_root):
    person_root = os.path.join(parent_root, person)
    print("Working on: ", person_root)
    counter = 0
    for instance in os.listdir(person_root):
        counter += 1
        name = 'person_{}_{}'.format(person, counter)
        print('Processing {} ...'.format(name))
        source_path = os.path.join(person_root, instance)
        print("Opening ", source_path)
        image = cv2.imread(source_path, cv2.COLOR_BGR2RGB)
        image_cp = image.copy()
        r, _ = extract_masks(image)
        if len(r['masks']) != 0 and len(r['rois']) != 0:
            for i in range(len(r["rois"])):
                mask = r["masks"][:, :, i].astype(int)
                masked_image = apply_mask(image_cp, mask)
                x1, y1 = r["rois"][i][0], r["rois"][i][1]
                x2, y2 = r["rois"][i][2], r["rois"][i][3]
                print('x1: {}, y1: {}, x2: {}, y2: {}'.format(x1, y1, x2
                                                              , y2))
                cropped_frame = crop_frame(x1, x2, y1, y2, masked_image)
                cropped_frame = lbp(cropped_frame)
                cv2.imshow("", cropped_frame)
                cv2.waitKey(0)

Working on:  Datasets/Propio/primer_grupo/test_data\1
Processing person_1_1 ...
Opening  Datasets/Propio/primer_grupo/test_data\1\001_9 16.jpg
x1: 94, y1: 149, x2: 703, y2: 338
Processing person_1_2 ...
Opening  Datasets/Propio/primer_grupo/test_data\1\001_9 17.jpg


KeyboardInterrupt: 